In [1]:
import os
from glob import glob

import main_fte
from py_utils import data_ops
import evaluate_fte
from lib import app
from py_utils import log

%load_ext autoreload
%autoreload 2

%matplotlib inline

# root_dir = os.path.join("..", "data")
root_dir = os.path.join("/Users/zico/OneDrive - University of Cape Town/CheetahReconstructionResults/cheetah_videos")
# If you are running the code remotely on the Linux i9.
# root_dir = os.path.join("/","data", "dlc", "to_analyse", "cheetah_videos")\
logger = log.logger("main")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# Generate the pose functions if this has not been done before (note this should only be performed once). It will be saved in the root_dir.
functions = main_fte.create_pose_functions(root_dir)

In [2]:
# Configurations for the optimisation. 2017_08_29/top/phantom/run1_1 2017_08_29/top/phantom/flick1_1 2019_02_27/kiara/run
# 2017_08_29/top/jules/run1_1
data_path = os.path.join("2017_08_29", "top", "jules", "run1_1")
# /Users/zico/OneDrive - University of Cape Town/CheetahReconstructionResults/paws-pw-included/2019_03_09/jules/flick2
start_frame = 10
end_frame = 110
dlc_thresh = 0.5

In [ ]:
# Run the optimisation
main_fte.run(root_dir, data_path, start_frame, end_frame, dlc_thresh, init_ekf=False, single_view=2, pairwise_included=0, generate_reprojection_videos=True, out_dir_prefix="/Users/zico/msc/dev/AcinoSet/data")

2021-09-24T08:22:02.662 | INFO | main_fte: Prepare data - Start


Loaded extrinsics from /Users/zico/OneDrive - University of Cape Town/CheetahReconstructionResults/cheetah_videos/2017_08_29/top/extrinsic_calib/6_cam_scene_sba.json



2021-09-24T08:22:05.752 | INFO | main_fte: Load H5 2D DLC prediction data


Found 2070 pairwise points between camera 0 and 1
Found 1729 pairwise points between camera 1 and 2
Found 1016 pairwise points between camera 2 and 3
Found 785 pairwise points between camera 3 and 4
Found 767 pairwise points between camera 4 and 5
Found 903 pairwise points between camera 5 and 0



2021-09-24T08:22:06.255 | INFO | main_fte: Estimate the initial trajectory
2021-09-24T08:22:06.971 | INFO | main_fte: Train motion prediction model
2021-09-24T08:22:07.054 | INFO | main_fte: X_train: (1119, 58), y_train: (1119, 29)
2021-09-24T08:22:07.057 | INFO | main_fte: X_test: (280, 58), y_test: (280, 29)
2021-09-24T08:22:07.240 | INFO | main_fte: Pure Python LR prediction matches sklearn :)
2021-09-24T08:22:07.246 | INFO | main_fte: Model prediction error: 0.17306254153641434
2021-09-24T08:22:07.252 | INFO | main_fte: Prepare data - End
2021-09-24T08:22:07.259 | INFO | main_fte: Start frame: 9, End frame: 110, Frame rate: 90
2021-09-24T08:22:07.263 | INFO | main_fte: Setup optimisation - Start
2021-09-24T08:22:07.502 | INFO | main_fte: Measurement initialisation...Done
2021-09-24T08:22:11.323 | INFO | main_fte: Variable initialisation...Done
2021-09-24T08:22:48.642 | INFO | main_fte: Constaint initialisation...Done
2021-09-24T08:22:49.238 | INFO | main_fte: Objective initialisati

...

  0%|          | 0/143 [00:00<?, ? f/s]

Total frames: 143 with frame dimensions: 1920 x 1080


  0%|          | 0/143 [00:00<?, ? f/s]

Generating frames and creating video...


  0%|          | 0/143 [00:00<?, ? f/s]

In [ ]:
dummy_scene = "/Users/zico/OneDrive - University of Cape Town/CheetahReconstructionResults/cheetah_videos/2017_08_29/top/extrinsic_calib/6_cam_scene_sba.json"
main_fte.plot_trajectory("/Users/zico/msc/dev/monocular_estimation/data/synthesized.pickle", dummy_scene)

In [ ]:
# Compare the optimisation with previous results i.e. compare the output pickle file fte.pickle. This is a visual inspection.abs
main_fte.plot_cheetah(root_dir, data_path, out_dir_prefix="/Users/zico/msc/dev/AcinoSet/data")

In [ ]:
# Compare the optimisation with previous results i.e. compare the output pickle file fte.pickle. This is a visual inspection.abs
main_fte.compare_cheetahs("/Users/zico/msc/dev/monocular_estimation/lily_discrete_trajectory.pickle", root_dir, data_path, out_dir_prefix="/Users/zico/msc/dev/AcinoSet/data")

In [ ]:
eval_dir = os.path.join("/Users/zico/msc/dev/AcinoSet/data", data_path, "fte_pw")
data = data_ops.load_pickle(os.path.join(eval_dir, "fte.pickle"))
results_dir = os.path.join(eval_dir, "evaluation")
os.makedirs(eval_dir, exist_ok=True)
max_delta_acc = evaluate_fte.eval_delta_acc(data, results_dir)
evaluate_fte.eval_model_error(data, results_dir)
evaluate_fte.eval_meas_error(data, results_dir)
logger.info(max_delta_acc)

In [ ]:
fig_fpath = os.path.join(os.path.join(os.path.join("..", "data"), data_path, "fte_pw"), 'fte.svg')
pw_file = data_ops.load_pickle(os.path.join(os.path.join("..", "data"), data_path, "fte_pw", "fte.pickle"))
app.plot_cheetah_states(pw_file["x"], out_fpath=fig_fpath)

In [ ]:
main_fte.plot_cost_functions()